In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2


import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
print_tensor = lambda n, x: print(n, type(x), x.shape, x.min(), x.max())
print(sys.path)
# from pathlib import Pathth


['/Users/monolith/Documents/git/mmseg4med/demo', '/Users/monolith/miniconda3/lib/python38.zip', '/Users/monolith/miniconda3/lib/python3.8', '/Users/monolith/miniconda3/lib/python3.8/lib-dynload', '', '/Users/monolith/.local/lib/python3.8/site-packages', '/Users/monolith/miniconda3/lib/python3.8/site-packages', '/Users/monolith/Documents/git/mmaction2', '/Users/monolith/Documents/git/nnDetection', '/Users/monolith/.local/lib/python3.8/site-packages/IPython/extensions', '/Users/monolith/.ipython', '/Users/monolith/Documents/git/mmseg4med']


In [3]:
# RibFrac101
import torch
import numpy as np
from skimage.transform import resize as skiresize
from pathlib import Path
from mmseg.datasets.pipelines.io4med import *

In [22]:
data_rt = Path('/Users/monolith/Desktop/ribfracture/processed/organize_raw/pub_ribfrac20')
caseid = 'RibFrac332'
image_fn = f'{caseid}/{caseid}_image.nii.gz'
image_raw, af = IO4Nii.read(data_rt/image_fn, verbose = True, dtype = np.int16)

NIBABEL


[IO4Nii] affine raw
 [[  -0.740234    0.          0.        190.344   ]
 [   0.         -0.740234    0.        189.5     ]
 [   0.          0.          1.25     -429.78    ]
 [   0.          0.          0.          1.      ]]
[IO4Nii] affine xyz
 [[  -0.740234    0.          0.        190.344   ]
 [   0.         -0.740234    0.        189.5     ]
 [   0.          0.          1.25     -429.78    ]
 [   0.          0.          0.          1.      ]]
[IO4Nii] xyz dim index [0, 1, 2]
[IO4Nii] xyz sign [1, 1, 1]
[IO4Nii] image raw <class 'numpy.ndarray'> float64 (512, 512, 349) -3024.0 3071.0
[IO4Nii] image xyz <class 'numpy.ndarray'> float64 (512, 512, 349) -3024.0 3071.0


In [23]:
shape_raw = np.array(image_raw.shape)
origin_spacing = np.array([abs(af[i, i]) for i in range(3)])

target_spacing = np.array([0.7, 0.7, 1.25])
shape_new = [round(a) for a in np.array(shape_raw * origin_spacing / target_spacing)]
print('spacing origin target', origin_spacing, target_spacing)
print('shape origin target', shape_raw, shape_new)
order = 3
# 0: Nearest-neighbor
# 1: Bi-linear (default)
# 2: Bi-quadratic
# 3: Bi-cubic
# 4: Bi-quartic
# 5: Bi-quintic

image_ski = skiresize(image_raw.astype(float), shape_new, order = order).astype(np.int16)

print_tensor('resize by ski', image_ski)
IO4Nii.write(image_ski, data_rt, image_fn.replace('image', f'image_skiresize_{order}'), affine_matrix= af)


spacing origin target [0.740234 0.740234 1.25    ] [0.7  0.7  1.25]
shape origin target [512 512 349] [541, 541, 349]
resize by ski <class 'numpy.ndarray'> int16 (541, 541, 349) -3024 3071


'/Users/monolith/Desktop/ribfracture/processed/organize_raw/pub_ribfrac20/RibFrac332/RibFrac332_image_skiresize_3.nii.gz.nii.gz'

In [24]:
from torch.nn.functional import interpolate as torch_interpolate
image_raw_tensor = torch.from_numpy(image_raw).float()[None, None]
# size=size, mode=mode, align_corners=align_corners
image_torch = torch_interpolate(image_raw_tensor, size = shape_new, mode = 'trilinear', align_corners = None)
image_torch = image_torch.cpu().short().numpy()[0, 0]
print_tensor('resize by torch', image_torch)
IO4Nii.write(image_torch, data_rt, image_fn.replace('image', 'image_torchresize'), affine_matrix= af)


resize by torch <class 'numpy.ndarray'> int16 (541, 541, 349) -3024 3071


'/Users/monolith/Desktop/ribfracture/processed/organize_raw/pub_ribfrac20/RibFrac332/RibFrac332_image_torchresize.nii.gz.nii.gz'

In [29]:
# Args:
#     width: sizes along width dimension (4.0, 6.0, 8.0)
#     height: sizes along height dimension (7.0, 11.0, 16.0)
#     depth: sizes along depth dimension (6.0, 9.0, 13.0)
width = (4, 6, 8)
height = (7, 11, 16)
depth = (6, 9, 13)
from itertools import product
def generate_anchors(width, #: Tuple[int],
                     height,  #: Tuple[int],
                     depth, #: Tuple[int],
                     dtype = torch.float, #: Union[torch.device, str]
                     device = "cpu"): #-> torch.Tensor
    """
    Generate anchors for given width, height and depth sizes

    Args:
        width: sizes along width dimension (4.0, 6.0, 8.0)
        height: sizes along height dimension (7.0, 11.0, 16.0)
        depth: sizes along depth dimension (6.0, 9.0, 13.0)

    Returns:
        Tensor: anchors of shape [n(width) * n(height) * n(depth) , dim * 2]
    """
    aa = product(width, height, depth)
    _ = [print(a) for a in aa]
    
    all_sizes = torch.tensor(list(product(width, height, depth)),
                             dtype=dtype, device=device) / 2
    print(all_sizes)
    anchors = torch.stack(
        [-all_sizes[:, 0], -all_sizes[:, 1], all_sizes[:, 0], all_sizes[:, 1],
         -all_sizes[:, 2], all_sizes[:, 2]], dim=1
        )
    print(anchors)
    return anchors

anchors = generate_anchors(width, height, depth)

(4, 7, 6)
(4, 7, 9)
(4, 7, 13)
(4, 11, 6)
(4, 11, 9)
(4, 11, 13)
(4, 16, 6)
(4, 16, 9)
(4, 16, 13)
(6, 7, 6)
(6, 7, 9)
(6, 7, 13)
(6, 11, 6)
(6, 11, 9)
(6, 11, 13)
(6, 16, 6)
(6, 16, 9)
(6, 16, 13)
(8, 7, 6)
(8, 7, 9)
(8, 7, 13)
(8, 11, 6)
(8, 11, 9)
(8, 11, 13)
(8, 16, 6)
(8, 16, 9)
(8, 16, 13)
tensor([[2.0000, 3.5000, 3.0000],
        [2.0000, 3.5000, 4.5000],
        [2.0000, 3.5000, 6.5000],
        [2.0000, 5.5000, 3.0000],
        [2.0000, 5.5000, 4.5000],
        [2.0000, 5.5000, 6.5000],
        [2.0000, 8.0000, 3.0000],
        [2.0000, 8.0000, 4.5000],
        [2.0000, 8.0000, 6.5000],
        [3.0000, 3.5000, 3.0000],
        [3.0000, 3.5000, 4.5000],
        [3.0000, 3.5000, 6.5000],
        [3.0000, 5.5000, 3.0000],
        [3.0000, 5.5000, 4.5000],
        [3.0000, 5.5000, 6.5000],
        [3.0000, 8.0000, 3.0000],
        [3.0000, 8.0000, 4.5000],
        [3.0000, 8.0000, 6.5000],
        [4.0000, 3.5000, 3.0000],
        [4.0000, 3.5000, 4.5000],
        [4.0000, 3.5000

In [ ]:
# 'width': [(4.0, 6.0, 8.0), (8.0, 12.0, 16.0), (16.0, 24.0, 32.0), (16.0, 24.0, 32.0)], 
# 'height': [(7.0, 11.0, 16.0), (14.0, 22.0, 32.0), (28.0, 44.0, 64.0), (56.0, 88.0, 128.0)], 
# 'depth': [(6.0, 9.0, 13.0), (12.0, 18.0, 26.0), (24.0, 36.0, 52.0), (48.0, 72.0, 104.0)], 

In [84]:
base_size = 4 #  = stride
ratios = torch.tensor([1/2, 1.0, 2])
center = 0
octave_base_scale = 2
scales_per_octave = 2
scale_major = True


octave_scales = np.array([2**(i / scales_per_octave) for i in range(scales_per_octave)])
scales = octave_scales * octave_base_scale
print('octave scale', octave_scales)
print('scale ', scales)

w = h = d = base_size
x_center = y_center = z_center = center

ratio_unit = torch.pow(ratios, 1/4)
ratio_units = torch.unique(torch.cat([ratio_unit, ratio_unit * ratio_unit]))[::2] # 
hwd_sizes = torch.tensor(list(product(ratio_units, ratio_units, ratio_units)))
hwd_prod = torch.prod(hwd_sizes, axis = -1).view(-1, 1)
hwd_sizes_nx4 = torch.cat([hwd_sizes, hwd_prod], axis = -1)
hwd_mask = torch.abs(hwd_sizes_nx4[:, -1] - 1 ) < 1e-5
hwd1sizes = hwd_sizes[hwd_mask]
# all_sizes = torch.tensor(list(product(width, height, depth))
# 1/u, 1/u, u^2
# 1/u, u, 1
# 1/u^2, u, u
# 1, 1, 1
print('ratio 1/4', ratio_unit)
print('ratio units', ratio_units)
# print('hwd sizes nx4', hwd_sizes_nx4)
print('hwd 1 area', hwd1sizes, hwd1sizes.shape)


h_ratios = hwd1sizes[:, 0]
w_ratios = hwd1sizes[:, 1]
d_ratios = hwd1sizes[:, 2]

# print(f'ratios {ratios} ratio unit {ratio_unit}')
print('h_ratios', h_ratios)
print('w_ratios', w_ratios)
print('d_ratios', d_ratios)

if scale_major:
    ws = (w * w_ratios[:, None] * scales[None, :]).view(-1)
    hs = (h * h_ratios[:, None] * scales[None, :]).view(-1)
    ds = (d * d_ratios[:, None] * scales[None, :]).view(-1)
else:
    ws = (w * scales[:, None] * w_ratios[None, :]).view(-1)
    hs = (h * scales[:, None] * h_ratios[None, :]).view(-1)
    ds = (d * scales[:, None] * d_ratios[None, :]).view(-1)

print('hs', hs)
print('ws', ws)
print('ds', ds)
# use float anchor and the anchor's center is aligned with the
# pixel center
base_anchors = [
    x_center - 0.5 * ws, y_center - 0.5 * hs, z_center - 0.5 * ds, 
    x_center + 0.5 * ws, y_center + 0.5 * hs, z_center + 0.5 * ds
]
base_anchors = torch.stack(base_anchors, dim=-1) # nx6
print(base_anchors)
print(base_anchors.shape)
        

octave scale [1.         1.41421356]
scale  [2.         2.82842712]
ratio 1/4 tensor([0.8409, 1.0000, 1.1892])
ratio units tensor([0.7071, 1.0000, 1.4142])
hwd 1 area tensor([[0.7071, 1.0000, 1.4142],
        [0.7071, 1.4142, 1.0000],
        [1.0000, 0.7071, 1.4142],
        [1.0000, 1.0000, 1.0000],
        [1.0000, 1.4142, 0.7071],
        [1.4142, 0.7071, 1.0000],
        [1.4142, 1.0000, 0.7071]]) torch.Size([7, 3])
h_ratios tensor([0.7071, 0.7071, 1.0000, 1.0000, 1.0000, 1.4142, 1.4142])
w_ratios tensor([1.0000, 1.4142, 0.7071, 1.0000, 1.4142, 0.7071, 1.0000])
d_ratios tensor([1.4142, 1.0000, 1.4142, 1.0000, 0.7071, 1.0000, 0.7071])
hs tensor([ 5.6569,  8.0000,  5.6569,  8.0000,  8.0000, 11.3137,  8.0000, 11.3137,
         8.0000, 11.3137, 11.3137, 16.0000, 11.3137, 16.0000],
       dtype=torch.float64)
ws tensor([ 8.0000, 11.3137, 11.3137, 16.0000,  5.6569,  8.0000,  8.0000, 11.3137,
        11.3137, 16.0000,  5.6569,  8.0000,  8.0000, 11.3137],
       dtype=torch.float64)
ds te

In [54]:
denom = 2**(1/4)

print(denom **2 , denom, 1/denom, 1/denom ** 2, 1/ 2**(1/2))

1.414213562373095 1.189207115002721 0.8408964152537146 0.7071067811865476 0.7071067811865475
